<a href="https://colab.research.google.com/github/salinator-hub/Dspy-/blob/main/Script-wrappedOnboardingTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serve the model using vLLM

In [ ]:
!pip install dspy-ai vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [22]:
# Run server in foreground
# !python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq

# Run server in the background
!nohup python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq > server.log 2>&1 &
# stdout is redirected to a file `server.log` using `> server.log`.
# We use a quantized model prepared using AWQ quantization

In [28]:
# Run this cell again and again to monitor the status of the server.
# The server can take a few mintues to start.
# Once the server has started, you will see logs such as this:
# INFO 02-10 07:16:43 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
!tail server.log

INFO 02-16 13:09:35 api_server.py:209] args: Namespace(host=None, port=8000, allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, served_model_name=None, chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, root_path=None, middleware=[], model='TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ', tokenizer=None, revision=None, tokenizer_revision=None, tokenizer_mode='auto', trust_remote_code=False, download_dir=None, load_format='auto', dtype='auto', kv_cache_dtype='auto', max_model_len=None, worker_use_ray=False, pipeline_parallel_size=1, tensor_parallel_size=1, max_parallel_loading_workers=None, block_size=16, seed=0, swap_space=4, gpu_memory_utilization=0.9, max_num_batched_tokens=None, max_num_seqs=256, max_paddings=256, disable_log_stats=False, quantization='awq', enforce_eager=False, max_context_len_to_capture=8192, disable_custom_all_reduce=False, enable_lora=False, max_loras=1, max_lora_rank=16, lo

In [37]:
# Once the server is up and running, this should work
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","object":"model","created":1708089023,"owned_by":"vllm","root":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","parent":null,"permission":[{"id":"modelperm-63a13a7078f94726b4affa9e03dd3116","object":"model_permission","created":1708089023,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

# DSPy: 𝗗eclarative 𝗦elf-improving Language 𝗣rograms

In [ ]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BayesianSignatureOptimizer, BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune


In [ ]:
lm = dspy.HFClientVLLM(model="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", port=8000, url="http://localhost")

dspy.settings.configure(lm=lm)

colbertv2 = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

# # NOTE: After you finish this notebook, you can use GPT-3.5 like this if you like.
# turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct')
# # In that case, make sure to configure lm=turbo below if you choose to do that.

dspy.settings.configure(rm=colbertv2)

In [ ]:
predict = dspy.Predict('question -> answer')

predict(question="What is the capital of  ancient India?")

In [49]:
%%writefile Emotion.py

#ALL THE IMPORTS
import matplotlib.pyplot as plt
import numpy as np
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune
from dspy.teleprompt import KNNFewShot
from dspy.predict.knn import KNN
import argparse
from dspy.teleprompt import BayesianSignatureOptimizer

#MODEL
model_name="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ"
lm = dspy.HFClientVLLM(model=model_name, port=8000, url="http://localhost")
dspy.settings.configure(lm=lm)

# Training dataset
train = [("I feel so sad when I think about the loss of my beloved pet.", "Sadness"),
         ("This achievement brings me immense joy and satisfaction.", "Joy"),
         ("I'm deeply in love with the beautiful scenery of this place.", "Love"),
         ("His rude behavior towards me makes me feel anger rising inside.", "Anger"),
         ("The thought of failing this exam fills me with fear and anxiety.", "Fear"),
         ("What a pleasant surprise to receive unexpected gifts from friends!", "Surprise"),
         ("Watching this heartwarming movie always brings tears to my eyes.", "Sadness"),
         ("The success of our project fills me with pride and happiness.", "Joy"),
         ("Being surrounded by loved ones fills my heart with warmth and love.", "Love"),
         ("Her disrespectful attitude towards others fills me with anger.", "Anger")]

# Development dataset
devset = [("The news of his passing brought overwhelming sadness to everyone.", "Sadness"),
       ("The unexpected visit from my old friend filled me with joy.", "Joy"),
       ("Expressing your love openly and honestly is the key to a happy relationship.", "Love"),
       ("His constant complaints and negativity make me feel annoyance and anger.", "Anger"),
       ("Walking alone in the dark alley sends shivers of fear down my spine.", "Fear"),
       ("Finding a handwritten letter from my childhood friend was a delightful surprise.", "Surprise"),
       ("Seeing abandoned animals on the street always fills me with sadness.", "Sadness"),
       ("The breathtaking view from the mountaintop filled me with awe and wonder.", "Joy"),
       ("The bond between a mother and her child is pure and unconditional love.", "Love"),
       ("His sarcastic remarks during the meeting filled me with frustration.", "Anger")]

# Convert the dataset into DSPy Examples
train = [dspy.Example(sentence=sentence, sentiment=sentiment).with_inputs('sentence') for sentence, sentiment in train]
devset = [dspy.Example(sentence=sentence, sentiment=sentiment).with_inputs('sentence') for sentence, sentiment in devset]

# Print the lengths of trainset and devset
#print(len(train), len(devset))

# Access an example from trainset and devset
train_example = train[0]
dev_example = devset[0]
# print(train_example.sentence)



def should_be_correct(sen, set):
  check=dspy.Predict("sentence -> sentiment")
  result=check(sentence=f"Is this {set}; a correct sentiment for {sen}. Return 0 for False and 1 for True.")
  if result.sentiment==0:
    return False
  else:
    return True

class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""

    sentence = dspy.InputField(desc = "Input sentence")
    sentiment = dspy.OutputField(desc = "Predicted Emotion in word")


class EmotionPipelineAssertions(dspy.Module):
    def __init__(self, assert_on=True):
        super().__init__()
        self.assert_on = assert_on
        self.Emotion_predicted = dspy.ChainOfThought(Emotion)

    def forward(self, sentence, **kwargs):
        predicted_Sentiment = self.Emotion_predicted(sentence=sentence)

        if self.assert_on:
            dspy.Suggest(
                should_be_correct(sentence, predicted_Sentiment),
                "Emotion is correct.",
                target_module=Emotion
            )

        return predicted_Sentiment


class Assess(dspy.Signature):
    """Assess the quality of the solution along the specified dimension."""

    assessed_text = dspy.InputField()
    assessment_sentence = dspy.InputField()
    assessment_sentiment = dspy.OutputField(desc="Only True or False")


def metric(gold, pred, trace=None):
    sentence, sentiment, emos = gold.sentence, gold.sentiment, pred.sentiment

    correctness = f"The given emos{emos} will be effective for this sentence {sentence}. The gold sentiment is {sentiment}"
    informative = f"For this sentence {sentence} is the given {emos} informative. The gold sentiment is {sentiment}"

    correct = dspy.ChainOfThought(Assess)(assessed_text=sentence, assessment_sentence=correctness)
    informative = dspy.ChainOfThought(Assess)(assessed_text=sentence, assessment_sentence=informative)

    correctness_score = correct.assessment_sentiment
    info_score = informative.assessment_sentiment
    score = 0

    if correctness_score == "True":
        score += 1

    if info_score == "True":
        score += 1

    return score / 2.0


# Define your devset and kwargs here
# Replace placeholders with your actual development dataset and any additional keyword arguments you need for evaluation

kwargs = dict(display_progress=True, display_table=5)

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate( devset = devset, num_threads=1, display_progress=True, display_table=5)

# Define function to evaluate pipeline
def evaluate_pipeline(model_name, assertion=True, bayesian=True):
    Emotion_predicted = EmotionPipelineAssertions()

    # Compile the pipeline with specified configurations
    if assertion and bayesian:
        compiled_pipeline = BayesianSignatureOptimizer(metric=metric, n=5, init_temperature=1.0).compile(Emotion_predicted, devset=devset, optuna_trials_num=5, max_bootstrapped_demos=3, max_labeled_demos=5, eval_kwargs=kwargs)
        config_label = "Optimized Pipeline"
    elif assertion and not bayesian:
        compiled_pipeline = Emotion_predicted
        config_label = "Without Bayesian"
    elif not assertion and bayesian:
        without_assertion=EmotionPipelineAssertions(assert_on=False)
        compiled_pipeline = BayesianSignatureOptimizer(metric=metric, n=5, init_temperature=1.0).compile(without_assertion, devset=devset, optuna_trials_num=5, max_bootstrapped_demos=3, max_labeled_demos=5, eval_kwargs=kwargs)
        config_label = "Without Assertion"
    else:
        without_assertion_and_bayesian=EmotionPipelineAssertions(assert_on=False)
        compiled_pipeline = without_assertion_and_bayesian
        config_label = "No Assertion and No Bayesian"

    # Evaluate the compiled pipeline
    eval_score = evaluate_on_hotpotqa(compiled_pipeline, metric=metric)
    return eval_score, config_label


# Run the evaluation for each configuration
def run_ablation_study(model_name, disable_assertion=False, disable_bayesian=False):
    scores = []
    config_labels = []

    for assertion in [True, False]:
        if disable_assertion and not assertion:
            continue
        for bayesian in [True, False]:
            if disable_bayesian and not bayesian:
                continue
            eval_score, config_label = evaluate_pipeline(model_name, assertion=assertion, bayesian=bayesian)
            scores.append(eval_score)
            config_labels.append(config_label)

    return scores, config_labels

def use_pipeline_as_user(model_name, sentence):
    Emotion_predicted = EmotionPipelineAssertions()
    result = Emotion_predicted(sentence=sentence)
    print("Predicted Emotion:", result.sentiment)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Pipeline for suggesting home remedies.")
    parser.add_argument("--assertion", type=bool, default=True, help="Whether to include assertion in the pipeline. Default: True")
    parser.add_argument("--bayesian", type=bool, default=True, help="Whether to use Bayesian optimization. Default: True")
    parser.add_argument("--run-ablation", action="store_true", help="Perform an ablation study.")
    parser.add_argument("--model-name", type=str, default="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", help="Name of the language model. Default: TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ")
    parser.add_argument("--user-question", type=str, help="Question to use the pipeline as a user.")
    parser.add_argument("--disable-assertion", action="store_true", help="Disable assertion in the pipeline during ablation study.")
    parser.add_argument("--disable-bayesian", action="store_true", help="Disable Bayesian optimization in the pipeline during ablation study.")
    args = parser.parse_args()

    if args.run_ablation:
        scores, config_labels = run_ablation_study(args.model_name, disable_assertion=args.disable_assertion, disable_bayesian=args.disable_bayesian)
        # Plotting the results
        x = np.arange(len(config_labels))
        plt.bar(x, scores)
        plt.xlabel('Configuration')
        plt.ylabel('Evaluation Score')
        plt.title('Ablation Study Results')
        plt.xticks(x, config_labels, rotation=45)
        plt.tight_layout()
        plt.show()
    elif args.user_sentence:
        use_pipeline_as_user(args.model_name, args.user_sentence)
    else:
        eval_score, config_label = evaluate_pipeline(args.model_name, assertion=args.assertion, bayesian=args.bayesian)
        print(f"Evaluation Score: {eval_score}, Configuration: {config_label}")


Overwriting Emotion.py


In [50]:
!python Emotion.py --run-ablation --disable-assertion

 30% 3/10 [00:40<01:34, 13.53s/it]
Bootstrapped 3 full traces after 4 examples in round 0.
 30% 3/10 [00:42<01:38, 14.13s/it]
Bootstrapped 3 full traces after 4 examples in round 0.
 30% 3/10 [00:39<01:33, 13.32s/it]
Bootstrapped 3 full traces after 4 examples in round 0.
 30% 3/10 [00:42<01:38, 14.04s/it]
Bootstrapped 3 full traces after 4 examples in round 0.
[I 2024-02-16 14:04:59,923] A new study created in memory with name: no-name-03467153-6b0b-4d7e-8cde-83ff2c82e7de
Average Metric: 8.5 / 10  (85.0): 100% 10/10 [02:13<00:00, 13.32s/it]
Average Metric: 8.5 / 10  (85.0%)
/usr/local/lib/python3.10/dist-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)




Assess the quality of the solution along the specified dimension.

---

Follow the following format.

Assessed Text: ${assessed_text}

Assessment Sentence: ${assessment_sentence}

Reasoning: Let's think step by step in order to

In [ ]:
!python Emotion.py --user_sentence "are you ok"